
# Hybrid Movie Recommender System


Importing required libraries

In [136]:
import pandas as pd
from math import sqrt
import numpy as np
import warnings
from sklearn.preprocessing import MinMaxScaler
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
%matplotlib inline

Reading csv files into pandas dataframes

In [137]:
movies_df = pd.read_csv(r"C:\Users\ADMIN\Documents\ml-latest-small\movies.csv")
ratings_df = pd.read_csv(r"C:\Users\ADMIN\Documents\ml-latest-small\ratings.csv")
print(movies_df.head())
print(ratings_df.head())

   movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantasy  
2                               Comedy|Romance  
3                         Comedy|Drama|Romance  
4                                       Comedy  
   userId  movieId  rating  timestamp
0       1        1     4.0  964982703
1       1        3     4.0  964981247
2       1        6     4.0  964982224
3       1       47     5.0  964983815
4       1       50     5.0  964982931


## Preprocessing


For the sake of simplicity remove the year from the __movie title__ column

In [138]:
movies_df['title'] = movies_df.title.str.replace('(\(\d\d\d\d\))', '', regex=True)
#Removing leading and trailing whitespaces around the title
movies_df['title'] = movies_df['title'].apply(lambda x: x.strip())
movies_df.head()

,movieId,title,genres
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji,Adventure|Children|Fantasy
2,3,Grumpier Old Men,Comedy|Romance
3,4,Waiting to Exhale,Comedy|Drama|Romance
4,5,Father of the Bride Part II,Comedy


Creating movies matrix with genre by splitting __genres__ column using __One Hot Encoding__

In [139]:
movies_df['genres'] = movies_df['genres'].apply(lambda x: x.split('|'))
movies_df.head()

,movieId,title,genres
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]"
1,2,Jumanji,"[Adventure, Children, Fantasy]"
2,3,Grumpier Old Men,"[Comedy, Romance]"
3,4,Waiting to Exhale,"[Comedy, Drama, Romance]"
4,5,Father of the Bride Part II,[Comedy]


In [140]:
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


Dropping __timestamp__ column from __rating_df__

In [141]:
ratings_df = ratings_df.drop('timestamp', axis=1)
ratings_df

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0
...,...,...,...
100831,610,166534,4.0
100832,610,168248,5.0
100833,610,168250,5.0
100834,610,168252,5.0


Handling missing values

In [142]:
movies_df.isna().sum()

movieId    0
title      0
genres     0
dtype: int64

In [143]:
ratings_df.isna().sum()

userId     0
movieId    0
rating     0
dtype: int64

# User Input

Input user's previously watched movies and its ratings to find out his/her preferences

In [144]:
user_history = [
            {'title':"17 Again", 'rating':5.0},
            {'title':'Night at the Museum: Battle of the Smithsonian', 'rating':3.5},
            {'title':'Hangover, The', 'rating':4.0},
            {'title':"Wolf of Wall Street, The", 'rating':4.5},
            {'title':"Iron Man 2", 'rating':3.5},
            {'title':"Predators", 'rating':4.0},
            {'title':"Escape Plan", 'rating':3.0}
         ] 
user_input_df = pd.DataFrame(user_history)
user_input_df

,title,rating
0,17 Again,5.0
1,Night at the Museum: Battle of the Smithsonian,3.5
2,"Hangover, The",4.0
3,"Wolf of Wall Street, The",4.5
4,Iron Man 2,3.5
5,Predators,4.0
6,Escape Plan,3.0


# Content-Based recommendation system


<img src="https://datasciencedojo.com/wp-content/uploads/content-based-recommendation-system-suggestions.webp" title="Content-Based recommender system" width=500>
<h4>Source: Data Science Dojo</h4>

Adding genres as columns in the dataframe

In [145]:
CB_movies_df = movies_df[:]
for i, row in movies_df.iterrows():
    for genre in row['genres']:
        CB_movies_df.at[i, genre] = 1
CB_movies_df = CB_movies_df.fillna(0)
CB_movies_df.head()

,movieId,title,genres,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,...,Horror,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir,(no genres listed)
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1.0,1.0,1.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,Jumanji,"[Adventure, Children, Fantasy]",1.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,Grumpier Old Men,"[Comedy, Romance]",0.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,Waiting to Exhale,"[Comedy, Drama, Romance]",0.0,0.0,0.0,1.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,Father of the Bride Part II,[Comedy],0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


To weight the genres we are merging __user_input_df__ and __input_movies__

In [146]:
input_movies = CB_movies_df[CB_movies_df['title'].isin(user_input_df['title'].tolist())]
#Merging user input with the dataset
input_movies = pd.merge(user_input_df, input_movies, how='inner', on='title')
input_movies

,title,rating,movieId,genres,Adventure,Animation,Children,Comedy,Fantasy,Romance,...,Horror,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir,(no genres listed)
0,17 Again,5.0,68135,"[Comedy, Drama]",0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Night at the Museum: Battle of the Smithsonian,3.5,68793,"[Action, Comedy, IMAX]",0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,"Hangover, The",4.0,69122,"[Comedy, Crime]",0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,"Wolf of Wall Street, The",4.5,106782,"[Comedy, Crime, Drama]",0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Iron Man 2,3.5,77561,"[Action, Adventure, Sci-Fi, Thriller, IMAX]",1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
5,Predators,4.0,79057,"[Action, Sci-Fi, Thriller]",0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,Escape Plan,3.0,105653,"[Action, Mystery, Thriller]",0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Dropping __genres__ column

In [147]:
input_movies.drop('genres', axis=1, inplace=True)

Creating user genre table from the above merged dataframe

In [148]:
user_genre_table = input_movies.drop(['title','rating','movieId'], axis=1)
user_genre_table

,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir,(no genres listed)
0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [149]:
input_movies['rating']

0    5.0
1    3.5
2    4.0
3    4.5
4    3.5
5    4.0
6    3.0
Name: rating, dtype: float64

Weighted Genre Matrix is generated using dot product

In [150]:
user_profile = user_genre_table.T.dot(input_movies['rating'])
user_profile

Adventure              3.5
Animation              0.0
Children               0.0
Comedy                17.0
Fantasy                0.0
Romance                0.0
Drama                  9.5
Action                14.0
Crime                  8.5
Thriller              10.5
Horror                 0.0
Mystery                3.0
Sci-Fi                 7.5
War                    0.0
Musical                0.0
Documentary            0.0
IMAX                   7.0
Western                0.0
Film-Noir              0.0
(no genres listed)     0.0
dtype: float64

Now we can use the genre table and user profile to recommend movies which satisfies user's preferences

In [151]:
#Retrieving genre_table for every movie and removies unnecessary fields
genre_table = CB_movies_df[:]
#To retain movie information we are setting movieId as index
genre_table = genre_table.set_index(genre_table['movieId'])
genre_table = genre_table.drop(['movieId','title','genres'], axis=1)
genre_table

,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir,(no genres listed)
movieId,,,,,,,,,,,,,,,,,,,,
1,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193581,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
193583,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
193585,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Finding weighted average of every movie and recommending the top 10 movies

In [152]:
recommended_movies = (genre_table*user_profile).sum(axis=1)
recommended_movies

movieId
1         20.5
2          3.5
3         17.0
4         26.5
5         17.0
          ... 
193581    31.0
193583    17.0
193585     9.5
193587    14.0
193609    17.0
Length: 9742, dtype: float64

Dividing it with profile sum to normalize the score

In [153]:
#Multiply the genres by the weights and then take the weighted average
recommended_movies = recommended_movies/(user_profile.sum())
recommended_movies.head()

movieId
1    0.254658
2    0.043478
3    0.211180
4    0.329193
5    0.211180
dtype: float64

Sorting the score to find the top 10 movies to recommend

In [154]:
recommended_movies.sort_values(ascending=False, inplace=True)
recommended_movies.head(10)

movieId
81132    0.819876
79132    0.745342
7007     0.739130
5027     0.739130
7235     0.739130
145      0.739130
20       0.739130
4719     0.739130
1432     0.739130
5628     0.739130
dtype: float64

In [155]:
recommended_movies.head(10).keys()

Index([81132, 79132, 7007, 5027, 7235, 145, 20, 4719, 1432, 5628], dtype='int64', name='movieId')

These are the top 10 recommended movies for the user based on his/her interests using Content-Based recommendation

In [156]:
CB_movies_df.loc[CB_movies_df['movieId'].isin(recommended_movies.head(10).index)]['title']

19                        Money Train
118                          Bad Boys
1103                            Metro
3460                    Osmosis Jones
3657                  Another 48 Hrs.
3989                           Wasabi
4693              Last Boy Scout, The
4843    Ichi the Killer (Koroshiya 1)
7372                        Inception
7441                           Rubber
Name: title, dtype: object

# Collaborative Filtering

In [157]:
CF_movies_df = movies_df[:]
CF_ratings_df = ratings_df.copy()

Finding the users who has seen the same movies as our target user

In [158]:
user_neighbors = CF_ratings_df[CF_ratings_df['movieId'].isin(input_movies.movieId)]

In [159]:
user_neighbors.head()

,userId,movieId,rating
254,2,106782,5.0
1246,10,106782,1.0
2136,18,69122,3.0
2159,18,77561,4.0
2213,18,106782,3.0


We group the __user_neighbors__ dataframe by userId and sort them based on the number of similar movies they watch

In [160]:
user_neighbors = user_neighbors.groupby(['userId'])

In [161]:
user_neighbors = sorted(user_neighbors, key=lambda x: len(x[1]), reverse=True)

In [162]:
user_neighbors = user_neighbors[:12]

Finding similarity between the users using Pearson Correlation Coefficient

<img src="https://study.com/cimages/multimages/16/begning.jpg">
<h5>Source: www.study.com</h5>

In [163]:
similarity_scores = {}
input_movies = input_movies.sort_values(by='movieId')
for user, group in user_neighbors:
    n_ratings = len(group)
    #Sorting the movies in the group for alignment with the input movies
    group = group.sort_values(by='movieId')
    buffer_movies = input_movies[input_movies['movieId'].isin(group.movieId)]
    buffer_user_ratings = buffer_movies['rating'].tolist()
    buffer_group_ratings = group['rating'].tolist()
    #Calculating pearson correlation coefficient
    xy = sum( i*j for i, j in zip(buffer_user_ratings, buffer_group_ratings)) - sum(buffer_user_ratings)*sum(buffer_group_ratings)/float(n_ratings)
    xx = sum([i**2 for i in buffer_user_ratings]) - pow(sum(buffer_user_ratings),2)/float(n_ratings)
    yy = sum([i**2 for i in buffer_group_ratings]) - pow(sum(buffer_group_ratings),2)/float(n_ratings)
    
    
    #If the denominator is different than zero, then divide, else, 0 correlation.
    if xx != 0 and yy != 0:
        similarity_scores[user[0]] = xy/sqrt(xx*yy)
    else:
        similarity_scores[user[0]] = 0


In [164]:
similarity_scores.items()

dict_items([(21, -0.8910421112136238), (249, 0.8882347881956829), (68, -0.6324555320336759), (177, -0.7385489458759964), (298, 0.38569460791993504), (305, -0.3490050304482666), (380, 0), (414, 0.0), (448, 0.8703882797784892), (509, 0.09901475429766744), (599, -0.5), (610, 0.8528028654224417)])

In [165]:
pearson_df = pd.DataFrame.from_dict(similarity_scores, orient='index')
pearson_df.columns = ['similarity_index']
pearson_df['userId'] = pearson_df.index
pearson_df.index = range(len(pearson_df))
pearson_df.head()

,similarity_index,userId
0,-0.891042,21
1,0.888235,249
2,-0.632456,68
3,-0.738549,177
4,0.385695,298


Finding the top 10 neighbors based on similarity score

In [166]:
top_neighbors=pearson_df.sort_values(by='similarity_index', ascending=False)[0:10]
top_neighbors

,similarity_index,userId
1,0.888235,249
8,0.870388,448
11,0.852803,610
4,0.385695,298
9,0.099015,509
6,0.000000,380
7,0.000000,414
5,-0.349005,305
10,-0.500000,599
2,-0.632456,68


To find weighted ratings we are adding __similarityIndex__ column to __CF_ratings_df__ dataframe

In [167]:
top_neighbors_matrix=top_neighbors.merge(CF_ratings_df, left_on='userId', right_on='userId', how='inner')
top_neighbors_matrix

,similarity_index,userId,movieId,rating
0,0.888235,249,1,4.0
1,0.888235,249,2,4.0
2,0.888235,249,19,3.5
3,0.888235,249,20,3.5
4,0.888235,249,32,5.0
...,...,...,...,...
13944,-0.632456,68,164179,3.5
13945,-0.632456,68,166528,4.5
13946,-0.632456,68,166635,3.0
13947,-0.632456,68,168252,4.0


Calculating weighted ratings by multiplying __rating__ and __similarityIndex__

In [168]:
#Multiplies the similarity by the user's ratings
top_neighbors_matrix['weighted_rating'] = top_neighbors_matrix['similarity_index']*top_neighbors_matrix['rating']
top_neighbors_matrix.head()

,similarity_index,userId,movieId,rating,weighted_rating
0,0.888235,249,1,4.0,3.552939
1,0.888235,249,2,4.0,3.552939
2,0.888235,249,19,3.5,3.108822
3,0.888235,249,20,3.5,3.108822
4,0.888235,249,32,5.0,4.441174


Finding weighted_rating by movie using groupBy

In [169]:

top_neighbors_movie_rating = top_neighbors_matrix.groupby('movieId').sum()[['similarity_index','weighted_rating']]
top_neighbors_movie_rating.columns = ['movie_similarity_index','movie_weighted_rating']
top_neighbors_movie_rating.head()

,movie_similarity_index,movie_weighted_rating
movieId,,
1,1.963680,10.255204
2,0.662857,2.304295
3,-0.262067,0.596254
5,0.336948,1.494776
6,-0.628658,-1.737325


Normalizing __movie_weighted_rating__ by dividing it with __movie_similarity_index__

In [170]:
top_neighbors_movie_rating['recommendation_score'] = top_neighbors_movie_rating['movie_weighted_rating']/top_neighbors_movie_rating['movie_similarity_index']
top_neighbors_movie_rating.head()

,movie_similarity_index,movie_weighted_rating,recommendation_score
movieId,,,
1,1.963680,10.255204,5.222442
2,0.662857,2.304295,3.476307
3,-0.262067,0.596254,-2.275194
5,0.336948,1.494776,4.436228
6,-0.628658,-1.737325,2.763548


Sort __top_neighbors_movie_rating__ to find top 10 movies to recommend

In [171]:
top_neighbors_movie_rating = top_neighbors_movie_rating.sort_values(by='recommendation_score', ascending=False)
top_neighbors_movie_rating.head(10)

,movie_similarity_index,movie_weighted_rating,recommendation_score
movieId,,,
4105,0.003798,1.367994,360.203699
110501,0.003798,1.367994,360.203699
106766,0.003798,1.213201,319.445529
7387,0.003798,0.443492,116.774871
4553,0.003798,0.439694,115.774871
968,0.003798,0.166186,43.758170
474,0.021383,0.835533,39.074183
63859,0.005789,0.194764,33.643914
89904,0.021383,0.661030,30.913472


These are the top 10 recommended movies for the user based on his/her neighbors using Collaborative-Filtering

In [172]:
CF_movies_df.loc[CF_movies_df['movieId'].isin(top_neighbors_movie_rating.head(10).index)]

,movieId,title,genres
412,474,In the Line of Fire,"[Action, Thriller]"
741,968,Night of the Living Dead,"[Horror, Sci-Fi, Thriller]"
2992,4007,Wall Street,[Drama]
3061,4105,"Evil Dead, The","[Fantasy, Horror, Thriller]"
3355,4553,They Live,"[Action, Sci-Fi, Thriller]"
4927,7387,Dawn of the Dead,"[Action, Drama, Horror]"
6903,63859,Bolt,"[Action, Adventure, Animation, Children, Comedy]"
7704,89904,The Artist,"[Comedy, Drama, Romance]"
8304,106766,Inside Llewyn Davis,[Drama]
8404,110501,The Raid 2: Berandal,"[Action, Crime, Thriller]"


In [173]:
final_recommended_movies = pd.merge(recommended_movies.to_frame() , top_neighbors_movie_rating, left_index=True, right_index=True)
final_recommended_movies.columns = ['movie_score', 'movie_similarity_index', 'movie_weighted_rating', 'recommendation_score']
final_recommended_movies['recommendation_score'] = final_recommended_movies['recommendation_score']/final_recommended_movies['recommendation_score'].max()

In [174]:
sorted_final_recommended_movies = final_recommended_movies.sort_values(by = ['movie_score', 'recommendation_score'], ascending = [False, True], na_position = 'last')
moviesIds = sorted_final_recommended_movies.head(10).index

### Final recommended movies using both Content-Based and Collaborative Filtering Recommender Systems

In [175]:
movies_df.loc[movies_df['movieId'].isin(moviesIds)]['title']

19                        Money Train
118                          Bad Boys
1103                            Metro
3460                    Osmosis Jones
3657                  Another 48 Hrs.
3989                           Wasabi
4693              Last Boy Scout, The
4843    Ichi the Killer (Koroshiya 1)
7372                        Inception
7441                           Rubber
Name: title, dtype: object